In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline

In [3]:
device = (torch.device("cuda" if torch.cuda.is_available() else "cpu"))

In [28]:
device = "cuda"

Loading the dataset

PyTorch's ```Normalize``` transform takes two arguments:
- ```mean```: a sequence of means
- ```std```: a sequence of standard deviations
where the sequence length should be the number of channels.

In [4]:
transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
train_dataset = datasets.CIFAR10(root="./data",
                                train=True,
                                transform=transform,
                                download=True)
test_dataset = datasets.CIFAR10(root="./data",
                               train=False,
                               transform=transform)

Files already downloaded and verified


Defining model hyperparameters and creating dataloaders

In [49]:
num_epochs = 4
batch_size = 32
lr = 1e-2

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                        batch_size=batch_size)

Model definition

In [37]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,
                              out_channels=6,
                              kernel_size=5,
                              stride=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2,
                                 stride=2)
        self.conv2 = nn.Conv2d(in_channels=6,
                              out_channels=16,
                              kernel_size=5,
                              stride=1)
        self.fc1 = nn.Linear(in_features=16*5*5,
                            out_features=120)
        self.fc2 = nn.Linear(in_features=120,
                            out_features=84)
        self.fc3 = nn.Linear(in_features=84,
                            out_features=10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

Defining loss and optimizer

In [50]:
model = ConvNet().to(device)

In [51]:
objective = nn.CrossEntropyLoss() # Do not apply softmax
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

Training loop

In [52]:
num_batches = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Conversion to correct shape
        # 32 x 3 x 32 x 32 -> 32 x 3 x 1024
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward Pass
        y_pred = model(images).to(device)
        loss = objective(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i % 100 == 0):
            print(
            f"Epoch:\t{epoch}\tbatch:\t{i}/{num_batches}\tloss:\t{loss.item()}")

Epoch:	0	batch:	0/1563	loss:	2.318211317062378
Epoch:	0	batch:	100/1563	loss:	2.300767183303833
Epoch:	0	batch:	200/1563	loss:	2.315434455871582
Epoch:	0	batch:	300/1563	loss:	2.3048276901245117
Epoch:	0	batch:	400/1563	loss:	2.294724702835083
Epoch:	0	batch:	500/1563	loss:	2.300926923751831
Epoch:	0	batch:	600/1563	loss:	2.2964396476745605
Epoch:	0	batch:	700/1563	loss:	2.2828571796417236
Epoch:	0	batch:	800/1563	loss:	2.2831249237060547
Epoch:	0	batch:	900/1563	loss:	2.281726598739624
Epoch:	0	batch:	1000/1563	loss:	2.2380683422088623
Epoch:	0	batch:	1100/1563	loss:	2.119163751602173
Epoch:	0	batch:	1200/1563	loss:	2.1538150310516357
Epoch:	0	batch:	1300/1563	loss:	1.9885618686676025
Epoch:	0	batch:	1400/1563	loss:	1.9433199167251587
Epoch:	0	batch:	1500/1563	loss:	1.9678658246994019
Epoch:	1	batch:	0/1563	loss:	2.1921746730804443
Epoch:	1	batch:	100/1563	loss:	2.092982530593872
Epoch:	1	batch:	200/1563	loss:	1.7823247909545898
Epoch:	1	batch:	300/1563	loss:	1.9456764459609985
Epoch:

Test loop

In [53]:
with torch.no_grad():
    n_samples = 0
    n_correct = 0
    n_class_correct = [0.0 for i in range(10)]
    n_class_samples = [0.0 for i in range(10)]
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        y_pred = model(images)
        vals, preds = torch.max(y_pred, dim=1)
        loss = objective(y_pred, labels)
        n_samples += labels.shape[0]
        n_correct += (preds == labels).sum().item()
        for j in range(labels.size()[0]):
            label = labels[j]
            pred = preds[j]
            if(label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    acc = n_correct / n_samples * 100.0
    print(f"Accuracy on test set: {acc}%")

Accuracy on test set: 46.9%
